<a href="https://colab.research.google.com/github/Haredkb/pGPP_LakesCONUS/blob/main/Pull_PARData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [49]:
# prompt:  find standard deviation and average PAR across all bands from 'MODIS/061/MCD18C2' for each point in df_in use corresponding date range test_datestart to test_dateend

#Corrected Code
import ee
import pandas as pd
from google.colab import drive


# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='par-lakes')

# Define the image collection
imageCollection = ee.ImageCollection('MODIS/061/MCD18C2')

# Define the points of interest
drive.mount('/content/drive')
df_in = pd.read_csv('/content/drive/My Drive/Projects/UNDERC/200_Data/Olesyk_Data/Olesyk_LakeMetadata_wdate.csv')
# Extract latitude and longitude to points
points = []
for index, row in df_in.iterrows():
    points.append(ee.Geometry.Point([row['Longitude_DD'], row['Latitude_DD']]))

# Extract start and end dates
time_ranges = []
for index, row in df_in.iterrows():
    start_date = row['start_date']
    end_date = row['end_date']
    time_ranges.append((start_date, end_date))

# Function to calculate average and standard deviation for each point and time range
def calculate_stats(point, start_date, end_date):
    filteredCollection = imageCollection.filterBounds(point).filterDate(start_date, end_date)

    def calculate_image_stats(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean().combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True),
            geometry=point,
            scale=500
        )
        return image.set(stats)

    stats_list = filteredCollection.map(calculate_image_stats).toList(filteredCollection.size())
    means = []
    stddevs = []
    for i in range(stats_list.size().getInfo()):
        stats_dict = ee.Dictionary(ee.Image(stats_list.get(i)).toDictionary()).getInfo()
        for band_name in stats_dict:
            if 'PAR' in band_name and 'mean' in band_name:
                means.append(stats_dict[band_name])
            elif 'PAR' in band_name and 'stdDev' in band_name:
                stddevs.append(stats_dict[band_name])

    if means:
      average_of_means = sum(means) / len(means) if means else None
      average_of_stddevs = sum(stddevs) / len(stddevs) if stddevs else None
      return average_of_means, average_of_stddevs
    else:
      return None, None

# Loop through points and time ranges, calculate stats
data = []
for i, (point, (start_date, end_date)) in enumerate(zip(points, time_ranges)):
    average, std_dev = calculate_stats(point, start_date, end_date)
    print(f"Point {i+1}, Time range: {start_date} - {end_date}")
    if average is not None:
        print(f"Average PAR: {average}")
        print(f"Standard Deviation of PAR: {std_dev}")
        data.append([i+1, point.coordinates().getInfo(), start_date, end_date, average, std_dev])
    else:
        print("No PAR data found for this point and time range.")

# Create a Pandas DataFrame
df = pd.DataFrame(data, columns=['Point', 'Coordinates', 'Start Date', 'End Date', 'Average PAR', 'Std Dev PAR'])

# Export the DataFrame to a CSV file in Google Drive
csv_file_path = '/content/drive/My Drive/Projects/UNDERC/200_Data/Olesyk_Data/point_stats_par.csv'
df.to_csv(csv_file_path, index=False)

print(f"DataFrame exported to: {csv_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Point 1, Time range: 2010-04-23 - 2010-09-29
Average PAR: 109.56558685910261
Standard Deviation of PAR: 0.0
Point 2, Time range: 2013-06-13 - 2013-09-13
Average PAR: 102.76714062269615
Standard Deviation of PAR: 0.0
Point 3, Time range: 2018-05-08 - 2018-11-06
Average PAR: 80.48762468603736
Standard Deviation of PAR: 0.0
Point 4, Time range: 2013-01-01 - 2013-12-31
Average PAR: 43.19794358245521
Standard Deviation of PAR: 0.0
Point 5, Time range: 2013-05-01 - 2013-11-30
Average PAR: 78.90820928771451
Standard Deviation of PAR: 0.0
Point 6, Time range: 2013-06-12 - 2013-11-24
Average PAR: 53.279846063721926
Standard Deviation of PAR: 0.0
Point 7, Time range: 2014-05-06 - 2014-10-16
Average PAR: 99.0064223931811
Standard Deviation of PAR: 0.0
Point 8, Time range: 2012-05-28 - 2012-10-26
Average PAR: 55.86426351651155
Standard Deviation of PAR: 0.0
Point 9, Time